# TRAIN #1

## Data Exploration

จากข้อมูลที่ได้มามี feature ทั้งหมด 23 feature และมี sample ทั้งหมด 10127 คน จากคำแนะนำของชุดข้อมูลได้ทำการลบ 2 feature สุดท้ายออก<br>
แยก feature Attrition_Flag ออกมาเก็บในตัวแปล df_y เหลือชุดข้อมูลในการวิเคราะห์ทั้งหมด 20 feature

In [1]:
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Image

df = pd.read_csv('credit_card_churn.csv',sep=',')
df.columns = ["CLIENTNUM","Attrition_Flag","Customer_Age","Gender","Dependent_count","Education_Level","Marital_Status","Income_Category","Card_Category","Months_on_book","Total_Relationship_Count","Months_Inactive_12_mon","Contacts_Count_12_mon","Credit_Limit","Total_Revolving_Bal","Avg_Open_To_Buy","Total_Amt_Chng_Q4_Q1","Total_Trans_Amt","Total_Trans_Ct","Total_Ct_Chng_Q4_Q1","Avg_Utilization_Ratio","Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1","Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2"]
df_y = df["Attrition_Flag"]
df = df.drop(["Attrition_Flag","Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1","Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2"],axis = 1)
print(df.shape)
df.head()

(10127, 20)


,CLIENTNUM,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,768805383,45,M,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,818770008,49,F,5,Graduate,Single,Less than $40K,Blue,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,713982108,51,M,3,Graduate,Married,$80K - $120K,Blue,36,4,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,769911858,40,F,4,High School,Unknown,Less than $40K,Blue,34,3,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,709106358,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,5,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000


โดยชุดข้อมูลของคนที่ใช้ Credit card อยู่ในปัจจุบันมีจำนวน 8500 คน และ ยกเลิก Credit card มีจำนวน 1627 คน<br>
และไม่มี missing value ในชุดข้อมูล

In [2]:
print(df_y.value_counts())
print("")
# ดู missing valus
print(df.isnull().sum())
print(df.shape)
# code handmake
# for i in range(0,len(df.columns)):
#     print(df.columns[i] +"\t\t"+ str(df[str(df.columns[i])].isnull().sum()))

Existing Customer    8500
Attrited Customer    1627
Name: Attrition_Flag, dtype: int64

CLIENTNUM                   0
Customer_Age                0
Gender                      0
Dependent_count             0
Education_Level             0
Marital_Status              0
Income_Category             0
Card_Category               0
Months_on_book              0
Total_Relationship_Count    0
Months_Inactive_12_mon      0
Contacts_Count_12_mon       0
Credit_Limit                0
Total_Revolving_Bal         0
Avg_Open_To_Buy             0
Total_Amt_Chng_Q4_Q1        0
Total_Trans_Amt             0
Total_Trans_Ct              0
Total_Ct_Chng_Q4_Q1         0
Avg_Utilization_Ratio       0
dtype: int64
(10127, 20)


# Data preprocessing

แยกชุดข้อมูลประเภท numeric and classification และเพื่ม y ที่เปลี่ยนข้อมูลจาก["Existing Customer","Attrited Customer"] เป็น [1,0] เข้าไปในข้อมูลทั้ง 2 ประเภท<br> 

In [3]:
# df_y = pd.get_dummies(df_y)
df_y = df_y.replace(["Existing Customer","Attrited Customer"],[1,0])
print(df_y.value_counts())


1    8500
0    1627
Name: Attrition_Flag, dtype: int64


In [4]:

df_class = df.select_dtypes(object)
df_numer = df.drop(df_class.columns,axis= 1)

# เพื่ม y 
df_class = df_class.join([df_y])
df_numer = df_numer.join([df_y])
print("numeric : "+str(df_numer.shape))
print("numeric : "+str(df_class.shape))
# df_numer = df.drop(df_class.columns)


numeric : (10127, 16)
numeric : (10127, 6)


one-hot ข้อมูลประเภท classification ในอยู่ในแบบใน numeric 

In [5]:
df_class = pd.get_dummies(df_class)
print(len(df_class.shape))
# print(len(df_class.columns))

2


Plot heatmap numeric

In [6]:
# corr = df_numer.corr()
# mask = np.array(corr)
# # print(corr)
# mask[np.tril_indices_from(mask)] = False     
# fig,ax = plt.subplots()
# fig.set_size_inches(10,5)
# sns.heatmap(corr,mask= mask,vmax =1,vmin = -1,annot= True,cmap= 'coolwarm')
Image(url="pic/heatmap numeric.png", width=300, height=300)

Plot heatmap classification

In [7]:
# corr = df_class.corr()
# mask = np.array(corr)
# # print(corr)
# mask[np.tril_indices_from(mask)] = False     
# fig,ax = plt.subplots()
# fig.set_size_inches(10,5)
# sns.heatmap(corr,mask= mask,vmax =1,vmin =-1,annot= True,cmap= 'coolwarm')
Image(url="pic/heatmap classification.png", width=300, height=300)

Pairplot numeric

In [8]:
# sns.pairplot(df_class,hue="Attrition_Flag")
Image(url="pic/pairplot classification.png", width=300, height=300)

Pairplot classification

In [9]:
# sns.pairplot(df_numer,hue="Attrition_Flag")
Image(url="pic/pairplot numeric.png", width=300, height=300)

จากการ pairplot ข้อมูลประเภท numeric มาว่ามีช่วงที่มีการแบ่งกลุ่มอย่างชัดเจนในช่วงสีแดง และสีฟ้า

In [10]:
Image(url="pic/pairplot highlight.png", width=300, height=300)

จากการ pairplot ข้อมูลประเภท numeric มาว่ามีช่วงที่มีการแบ่งกลุ่มอย่างชัดเจนในช่วงสีม่วง และสีฟ้า

ผมจึงได้ตัดใน feature ในข้อมูลประเภท numeric
* Credit_Limit 
* Total_Revolving_Bal
* Avg_Open_To_Buy
* Total_Amt_Chng_Q4_Q1
* Total_Trans_Amt
* Total_Trans_Ct
* Total_Ct_Chng_Q4_Q1
* Avg_Utilization_Ratio
* Attrition_Flag

In [11]:
df_numer = df_numer.drop([
    'CLIENTNUM', 
    'Customer_Age', 
    'Dependent_count', 
    'Months_on_book',
    'Total_Relationship_Count', 
    'Months_Inactive_12_mon',
    'Contacts_Count_12_mon'],axis = 1)
print(df_numer.shape)
print(df_numer.columns)

(10127, 9)
Index(['Credit_Limit', 'Total_Revolving_Bal', 'Avg_Open_To_Buy',
       'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Trans_Ct',
       'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio', 'Attrition_Flag'],
      dtype='object')


จากการทำ heatmap ข้อมูลประเภท classification พบว่ามีการแบ่งกลุ่มอย่างชัดเจนในช่วงสีแดง

In [12]:
Image(url="pic/heatmap highlight.png", width=300, height=300)

ผมจึงได้เลือกใช้ใน feature ในข้อมูลประเภท classificaltion
* Gender
* Income_Category

In [13]:

df_class = pd.get_dummies(df[["Gender","Income_Category"]])
print("df_class : "+str(df_class.shape))

df_full = df_numer.join([df_class])
df_full = df_full.drop("Attrition_Flag",axis=1)
print(df_full.shape)
print(df_full.columns)

df_class : (10127, 8)
(10127, 16)
Index(['Credit_Limit', 'Total_Revolving_Bal', 'Avg_Open_To_Buy',
       'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Trans_Ct',
       'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio', 'Gender_F', 'Gender_M',
       'Income_Category_$120K +', 'Income_Category_$40K - $60K',
       'Income_Category_$60K - $80K', 'Income_Category_$80K - $120K',
       'Income_Category_Less than $40K', 'Income_Category_Unknown'],
      dtype='object')


## Model Building

Decision tree

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_full,df_y,test_size = 0.25,random_state = 25)
print(y_train.value_counts())

print("X_train : " +str(X_train.shape))

1    6364
0    1231
Name: Attrition_Flag, dtype: int64
X_train : (7595, 16)


In [15]:
from sklearn.tree import DecisionTreeClassifier
model_DecisionTree = DecisionTreeClassifier(criterion='entropy',)
model_DecisionTree 

DecisionTreeClassifier(criterion='entropy')

In [16]:
# train model
model_DecisionTree.fit(X_train, y_train)
# predict model train
y_pred = model_DecisionTree.predict(X_train)

# Classification report
from sklearn.metrics import classification_report,confusion_matrix
print("Classification report \n=======================")
print(classification_report(y_true=y_train, y_pred=y_pred))
print("Confusion matrix \n=======================")
print(confusion_matrix(y_true=y_train, y_pred=y_pred))


Classification report 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1231
           1       1.00      1.00      1.00      6364

    accuracy                           1.00      7595
   macro avg       1.00      1.00      1.00      7595
weighted avg       1.00      1.00      1.00      7595

Confusion matrix 
[[1231    0]
 [   0 6364]]


In [17]:
y_pred = model_DecisionTree.predict(X_test)

print("Classification report \n=======================")
print(classification_report(y_true=y_test, y_pred=y_pred))
print("Confusion matrix \n=======================")
print(confusion_matrix(y_true=y_test, y_pred=y_pred))

Classification report 
              precision    recall  f1-score   support

           0       0.80      0.79      0.80       396
           1       0.96      0.96      0.96      2136

    accuracy                           0.94      2532
   macro avg       0.88      0.88      0.88      2532
weighted avg       0.94      0.94      0.94      2532

Confusion matrix 
[[ 313   83]
 [  77 2059]]


Decision tree